<a href="https://colab.research.google.com/github/MathMachado/DSWP/blob/master/Notebooks/revisao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install as libraries necessárias

In [ ]:
!pip install pycaret

In [ ]:
!pip install shap

### Carregar as libraries

In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt # Gráficos

import seaborn as sns
sns.set_theme(style = 'ticks', palette = 'pastel')

from pycaret import regression, classification # Canivete suiço para Machine Learning
import shap

from collections import Counter

import tensorflow as tf
tf.__version__

In [ ]:
from pycaret.utils import enable_colab
enable_colab()

### Configurações de ambiente

In [ ]:
pd_options = pd.options.display # Objeto para configuração do ambiente: número de linhas, colunas, precisão e etc

In [ ]:
pd_options.max_columns = 50 # número máximo de colunas
pd_options.max_rows = 50 # número máximo de linhas
pd_options.precision = 3 # número de casas decimais
pd_options.float_format = "{:,.3f}".format

### Facets - DataViz

In [ ]:
from IPython.core.display import display, HTML
import base64

In [ ]:
!pip install facets-overview==1.0.0

In [ ]:
from facets_overview.feature_statistics_generator import FeatureStatisticsGenerator

### Carregar os dados de treinamento e teste

In [ ]:
url_T = 'https://raw.githubusercontent.com/MathMachado/DataFrames/master/train_LABDATA.csv';
url_V = 'https://raw.githubusercontent.com/MathMachado/DataFrames/master/test_LABDATA.csv';

In [ ]:
df_T = pd.read_csv(url_T, index_col = 'id')
df_V = pd.read_csv(url_V, index_col = 'id')

In [ ]:
df_T.head()

### Tipos dos dados

In [ ]:
df_T.dtypes

### Conversão de tipos
#### TotalCharges
* A variável TotalCharges é do tipo 'object'. Entretanto, esta variável deveria ser do tipo 'float'.         

In [ ]:
#df_T['TotalCharges2'] = pd.to_numeric(df_T['TotalCharges']) # NÃO VAI FUNCIONAR, POIS indice 161 tem um valor NULL, o que impede da transformação funcionar correctamente.

dado que temos valores NULL na variável TotalCharges, então precisamos usar errors = 'coerce'. Vamos olhar para o índice 161 que é uma das linhas que inviabiliza o uso do pd.astype(np.float):

In [ ]:
df_T.iloc[161] # Como estamos interessados na linha de índice 161, então usamos df.iloc[]

In [ ]:
df_T['TotalCharges2'] = pd.to_numeric(df_T['TotalCharges'], errors = 'coerce')
df_V['TotalCharges2'] = pd.to_numeric(df_V['TotalCharges'], errors = 'coerce') # A mesma operação/transformação precisa ser feita em df_V:

df_T.dtypes

In [ ]:
df_T[['TotalCharges2', 'TotalCharges']].head(20)

In [ ]:
df_T.iloc[161]

Me parece que a conversão foi bem-sucedida.

Next: deletar a variável original.

In [ ]:
df_T = df_T.drop(columns = ['TotalCharges'], axis = 1)
df_V = df_V.drop(columns = ['TotalCharges'], axis = 1) # Mesma transformação na validação

In [ ]:
df_T.dtypes

#### SeniorCitizen
* A variável SeniorCitizen é do tipo 'int64'. Entretanto, esta variável deveria ser do tipo categórica.

In [ ]:
Counter(df_T['SeniorCitizen'])

In [ ]:
d_SeniorCitizen = {0 : 'No', 1: 'Yes'}

In [ ]:
df_T['SeniorCitizen2'] = df_T['SeniorCitizen'].map(d_SeniorCitizen)
df_V['SeniorCitizen2'] = df_V['SeniorCitizen'].map(d_SeniorCitizen) # Mesma transformação em teste

Counter(df_T['SeniorCitizen2'])

Alternativamente, poderíamos ter usado objeto.value_counts():

In [ ]:
df_T['SeniorCitizen2'].value_counts()

In [ ]:
df_T = df_T.drop(columns = ['SeniorCitizen'], axis = 1)
df_V = df_V.drop(columns = ['SeniorCitizen'], axis = 1)

### Feature Engineering

In [ ]:
df_T.head()

#### Construção da variável qtd_servicos_adicionais

In [ ]:
df_T["qtd_servicos_adicionais"] = 0
df_T.loc[df_T["OnlineSecurity"] == "Yes","qtd_servicos_adicionais"] = df_T["qtd_servicos_adicionais"] + 1
df_T.loc[df_T["OnlineBackup"] == "Yes","qtd_servicos_adicionais"] =  df_T["qtd_servicos_adicionais"] + 1
df_T.loc[df_T["DeviceProtection"] == "Yes","qtd_servicos_adicionais"] =  df_T["qtd_servicos_adicionais"] + 1
df_T.loc[df_T["TechSupport"] == "Yes","qtd_servicos_adicionais"] = df_T["qtd_servicos_adicionais"] + 1
df_T.loc[df_T["PhoneService"] == "Yes", "qtd_servicos_adicionais"] = df_T["qtd_servicos_adicionais"] + 1
df_T.loc[df_T["StreamingTV"] == "Yes", "qtd_servicos_adicionais"] = df_T["qtd_servicos_adicionais"] + 1
df_T.loc[df_T["StreamingMovies"] == "Yes", "qtd_servicos_adicionais"] = df_T["qtd_servicos_adicionais"] + 1
df_T.loc[df_T["InternetService"] == "Yes", "qtd_servicos_adicionais"] = df_T["qtd_servicos_adicionais"] + 1

A mesma variável foi criada abaixo no dataframe de teste/validação:

In [ ]:
df_V["qtd_servicos_adicionais"] = 0
df_V.loc[df_V["OnlineSecurity"] == "Yes","qtd_servicos_adicionais"] = df_V["qtd_servicos_adicionais"] + 1
df_V.loc[df_V["OnlineBackup"] == "Yes","qtd_servicos_adicionais"] =  df_V["qtd_servicos_adicionais"] + 1
df_V.loc[df_V["DeviceProtection"] == "Yes","qtd_servicos_adicionais"] =  df_V["qtd_servicos_adicionais"] + 1
df_V.loc[df_V["TechSupport"] == "Yes","qtd_servicos_adicionais"] = df_V["qtd_servicos_adicionais"] + 1
df_V.loc[df_V["PhoneService"] == "Yes", "qtd_servicos_adicionais"] = df_V["qtd_servicos_adicionais"] + 1
df_V.loc[df_V["StreamingTV"] == "Yes", "qtd_servicos_adicionais"] = df_V["qtd_servicos_adicionais"] + 1
df_V.loc[df_V["StreamingMovies"] == "Yes", "qtd_servicos_adicionais"] = df_V["qtd_servicos_adicionais"] + 1
df_V.loc[df_V["InternetService"] == "Yes", "qtd_servicos_adicionais"] = df_V["qtd_servicos_adicionais"] + 1

In [ ]:
df_T['qtd_servicos_adicionais'].describe()

In [ ]:
df_T['qtd_servicos_adicionais'].hist()

In [ ]:
Counter(df_T['qtd_servicos_adicionais'])

In [ ]:
#df_T = df_T.drop(columns = ['qtd_servicos_adicionais'], axis = 1)
#df_V = df_V.drop(columns = ['qtd_servicos_adicionais'], axis = 1)

In [ ]:
df_T.head()

#### Feature Engineering - Construção da variável tempo_contrato

In [ ]:
df_T[['tenure', 'Contract']].head()

In [ ]:
Counter(df_T['Contract'])

Como a variável 'Contract' não é numérica, fiz da seguinte forma:

In [ ]:
df_T.loc[df_T['Contract'] == 'Month-to-month', 'tempo_contrato'] = df_T['tenure']/1
df_T.loc[df_T['Contract'] == 'One year', 'tempo_contrato'] = df_T['tenure']/12
df_T.loc[df_T['Contract'] == 'Two year', 'tempo_contrato'] = df_T['tenure']/24

# Mesmas transformações no dataframe de teste:
df_V.loc[df_V['Contract'] == 'Month-to-month', 'tempo_contrato'] = np.round(df_V['tenure']/1, 1)
df_V.loc[df_V['Contract'] == 'One year', 'tempo_contrato'] = np.round(df_V['tenure']/12, 1)
df_V.loc[df_V['Contract'] == 'Two year', 'tempo_contrato'] = np.round(df_V['tenure']/24, 1)

In [ ]:
df_T[['tenure', 'Contract', 'tempo_contrato']].head()

In [ ]:
df_T['tempo_contrato'].hist(bins=10)

Decidi construir 10 classes para a variável tempo_contrato. Segue abaixo:
* Experimentando pd.qcut()

In [ ]:
df_T['tempo_contrato_qcut'] = pd.qcut(df_T['tempo_contrato'], q = 10, labels = ['0_1', '1_2', '2_3', '3_4', '4_5', '5_6', '6_7', '7_8', '8_9', '9_10'])
Counter(df_T['tempo_contrato_qcut'])

Mesma transformação no dataframe de teste:

In [ ]:
df_V['tempo_contrato_qcut'] = pd.qcut(df_V['tempo_contrato'], q = 10, labels = ['0_1', '1_2', '2_3', '3_4', '4_5', '5_6', '6_7', '7_8', '8_9', '9_10'])
Counter(df_V['tempo_contrato_qcut'])

* Experimentando pd.cut()

In [ ]:
#df_T['tempo_contrato_cut'] = pd.cut(df_T['tempo_contrato'], bins = 10)

bins_T = [-0.072, 7.2, 14.4, 21.6, 28.8, 36.0, 43.2, 50.4, 57.6, 64.8, 72.0]
labels_T = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'] # ['(-0.072, 7.2]', '(7.2, 14.4]', '(14.4, 21.6]', '(21.6, 28.8]', '(28.8, 36.0]', '(36.0, 43.2]', '(43.2, 50.4]', '(50.4, 57.6]', '(57.6, 64.8]', '(64.8, 72.0]']
                                                                                                                                                     
df_T['tempo_contrato_cut'] = pd.cut(df_T['tempo_contrato'], bins = bins_T, labels = labels_T)
Counter(df_T['tempo_contrato_cut'])

In [ ]:
df_T[['tempo_contrato_cut']].head()

A transformação no dataframe de teste é um pouco mais complicada: temos que nos assegurar que os pontos de corte no dataframe de validação sejam os mesmos do dataframe de treinamento.

In [ ]:
bins_T = [-0.072, 7.2, 14.4, 21.6, 28.8, 36.0, 43.2, 50.4, 57.6, 64.8, 72.0]
labels_T = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'] # ['(-0.072, 7.2]', '(7.2, 14.4]', '(14.4, 21.6]', '(21.6, 28.8]', '(28.8, 36.0]', '(36.0, 43.2]', '(43.2, 50.4]', '(50.4, 57.6]', '(57.6, 64.8]', '(64.8, 72.0]']
                                                                                                                                                     
df_V['tempo_contrato_cut'] = pd.cut(df_V['tempo_contrato'], bins = bins_T, labels = labels_T)
df_V[['tempo_contrato_cut']].head()

In [ ]:
Counter(df_T['tempo_contrato_cut'])

In [ ]:
Counter(df_V['tempo_contrato_cut'])

Como podemos ver, temos os mesmos pontos de corte para os dois dataframes: treinamento e teste.

A seguir, variáveis dummies:

In [ ]:
df_T.head()

In [ ]:
df_T = pd.get_dummies(df_T, columns = ['tempo_contrato_qcut'], dummy_na = True)
df_V = pd.get_dummies(df_V, columns = ['tempo_contrato_qcut'], dummy_na = True) # Mesma transformação no dataframe de teste

In [ ]:
df_T.head()

In [ ]:
df_T = pd.get_dummies(df_T, columns = ['tempo_contrato_cut'], dummy_na = True)
df_V = pd.get_dummies(df_V, columns = ['tempo_contrato_cut'], dummy_na = True) # Mesma transformação no dataframe de teste

A seguir, deletar as variáveis originais:

In [ ]:
df_T = df_T.drop(columns = ['tempo_contrato'], axis = 1)
df_V = df_V.drop(columns = ['tempo_contrato'], axis = 1)

In [ ]:
df_T.head()

#### Feature Engineering - Binnarizar a variável MonthlyCharges
* Vou tentar pd.qcut()

Decidi construir 10 classes para a variável MonthlyCharges usando pd.qcut(). Segue abaixo:

In [ ]:
df_T['MonthlyCharges_qcut'] = pd.qcut(df_T['MonthlyCharges'], q = 10, labels = ['0_1', '1_2', '2_3', '3_4', '4_5', '5_6', '6_7', '7_8', '8_9', '9_10'])
Counter(df_T['MonthlyCharges_qcut'])

Mesma transformação no dataframe de teste:

In [ ]:
df_V['MonthlyCharges_qcut'] = pd.qcut(df_V['MonthlyCharges'], q = 10, labels = ['0_1', '1_2', '2_3', '3_4', '4_5', '5_6', '6_7', '7_8', '8_9', '9_10'])
Counter(df_V['MonthlyCharges_qcut'])

A seguir, variáveis dummies:

In [ ]:
df_T = pd.get_dummies(df_T, columns = ['MonthlyCharges_qcut'], dummy_na = True)
df_V = pd.get_dummies(df_V, columns = ['MonthlyCharges_qcut'], dummy_na = True) # Mesma transformação no dataframe de teste

A seguir, deletar as variáveis originais:

In [ ]:
df_T = df_T.drop(columns = ['MonthlyCharges'], axis = 1)
df_V = df_V.drop(columns = ['MonthlyCharges'], axis = 1)

A seguir, listas por tipos de variáveis:

In [ ]:
l_features_numericas = list(df_T.select_dtypes([np.float64, np.int64]))
l_features_numericas.remove('Churn')
l_features_numericas

In [ ]:
l_features_categoricas = list(df_T.select_dtypes(np.object))
l_features_categoricas

### DataViz com Facets

In [ ]:
def mostra_dados(df):
    fsg = FeatureStatisticsGenerator()
    dataframes = [{'table': df, 'name': 'dados de treinamento'}]

    censusProto = fsg.ProtoFromDataFrames(dataframes)
    protostr = base64.b64encode(censusProto.SerializeToString()).decode("utf-8")

    HTML_TEMPLATE = """<script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
        <link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/1.0.0/facets-dist/facets-jupyter.html">
        <facets-overview id="elem"></facets-overview>
        <script>
          document.querySelector("#elem").protoInput = "{protostr}";
        </script>"""
    html = HTML_TEMPLATE.format(protostr=protostr)
    display(HTML(html))

In [ ]:
mostra_dados(df_T)

In [ ]:
df_T.columns

In [ ]:
l_colunas_booleanas = ['tempo_contrato_qcut_0_1', 'tempo_contrato_qcut_1_2',
       'tempo_contrato_qcut_2_3', 'tempo_contrato_qcut_3_4',
       'tempo_contrato_qcut_4_5', 'tempo_contrato_qcut_5_6',
       'tempo_contrato_qcut_6_7', 'tempo_contrato_qcut_7_8',
       'tempo_contrato_qcut_8_9', 'tempo_contrato_qcut_9_10',
       'tempo_contrato_qcut_nan', 'tempo_contrato_cut_A',
       'tempo_contrato_cut_B', 'tempo_contrato_cut_C', 'tempo_contrato_cut_D',
       'tempo_contrato_cut_E', 'tempo_contrato_cut_F', 'tempo_contrato_cut_G',
       'tempo_contrato_cut_H', 'tempo_contrato_cut_I', 'tempo_contrato_cut_J',
       'tempo_contrato_cut_nan', 'MonthlyCharges_qcut_0_1',
       'MonthlyCharges_qcut_1_2', 'MonthlyCharges_qcut_2_3',
       'MonthlyCharges_qcut_3_4', 'MonthlyCharges_qcut_4_5',
       'MonthlyCharges_qcut_5_6', 'MonthlyCharges_qcut_6_7',
       'MonthlyCharges_qcut_7_8', 'MonthlyCharges_qcut_8_9',
       'MonthlyCharges_qcut_9_10', 'MonthlyCharges_qcut_nan']

In [ ]:
d_transforma = {0: 'No', 1: 'Yes'}

for coluna in l_colunas_booleanas:
    df_T[coluna+'_2'] = df_T[coluna].map(d_transforma)    

In [ ]:
df_T = df_T.drop(columns = l_colunas_booleanas, axis = 1)

A seguir, a mesma transformação no dataframe de teste:

In [ ]:
for coluna in l_colunas_booleanas:
    if coluna in df_V.columns: # Flávia, isso evita erros caso a coluna não exista em df_V.
        df_V[coluna+'_2'] = df_V[coluna].map(d_transforma)
    else:
        l_colunas_booleanas.remove(coluna)

In [ ]:
df_V = df_V.drop(columns = l_colunas_booleanas, axis = 1)

In [ ]:
df_T.head()

In [ ]:
df_T2 = df_T.copy()
df_V2 = df_V.copy()

In [ ]:
mostra_dados(df_T2)

### Tratamento dos Missing Values
* Como vamos tratar os missing values?
    * **Proposição**: Construir modelos preditivos para estimar os valores missing em função das demais variáveis que NÃO SÃO missing.
    * Variáveis:
        * **Numéricas**:
            * TotalCharges2 (0.14% de missing values);
            * tenure (8.18% de missing values).
        * **Categóricas**:
            * PaymentMethod (1.76% de missing values);
            * Dependents (3.87% de missing values);

A seguir, funções para nos ajudar a construir os modelos de Machine Learning para estimar os missing values:

A função prepara_dataframes (abaixo) vai construir (automaticamente) 4 dataframes para, na sequência, desenvolvermos modelos de Machine Learning para tratamento dos missing values:
* df_TotalCharges2_sem_mv
* df_tenure_sem_mv
* df_PaymentMethod_sem_mv
* df_Dependents_sem_mv


In [ ]:
def prepara_dataframes(df_T, df_V, target, l_colunas_mv):
    df_mv = 'df' + '_' + target + '_sem_mv'
    #print(df2)

    l_colunas_mv.remove(target)
    print(f'Features ignoradas: {l_colunas_mv}')

    df_mv = df_T.dropna() # Excluir todas as linhas com missing values
    df_mv = df_mv.drop(columns = ['Churn'], axis = 1)

    # Para reduzir o viés, vamos dropar/deletar as features que são missing values
    df_mv = df_mv.drop(columns = l_colunas_mv, axis = 1)
    #print(df_mv.isna().sum())

    # Apontar os missing values no dataframe original:
    df_T[target+'_mv'] = np.where(df_T[target].isna(), 'Yes', 'No')
    df_V[target+'_mv'] = np.where(df_V[target].isna(), 'Yes', 'No')

    return df_mv, df_T, df_V

In [ ]:
l_colunas_mv = ['TotalCharges2', 'tenure', 'PaymentMethod', 'Dependents']

for target in l_colunas_mv:
    l_colunas_mv_2 = ['TotalCharges2', 'tenure', 'PaymentMethod', 'Dependents']
    exec(f"df_{target}_sem_mv, df_T3, df_V3 = prepara_dataframes(df_T2, df_V2, target, l_colunas_mv_2)")

Vamos verificar se os dataframes foram construídos:

In [ ]:
l_dataframes = [df_TotalCharges2_sem_mv, df_tenure_sem_mv, df_PaymentMethod_sem_mv, df_Dependents_sem_mv]

for dataframe in l_dataframes:
    print(dataframe.shape)

Quais foram as mudanças no dataframe original?
* A única alteração é a construção de 4 colunas intituladas:
    * TotalCharges2_mv;
    * tenure_mv;
    * PaymentMethod_mv;
    * Dependents_mv.

In [ ]:
df_T2.head()

A seguir, a função pycaret_regressao() e pycaret_classificacao() para nos ajudar a encontrar modelos de Machine Learning de Regressão e Classificação, respectivamente:

In [ ]:
def pycaret_regressao(df, target):
    print(f'Modelo: ml_{target}')
    print(f'Este é um problema de Regressão')

    ml = regression.setup(data = df, 
                          target = target, # esta variável é numérica. Portanto, estamos diante de um problema de regressão
                          session_id = 20111974,
                          #ignore_features = ['id'],
                          #numeric_features = l_features_numericas, # Necesssário para forçar o Pycaret a considerar estas features como numéricas
                          #ignore_low_variance = True,
                          #pca = True, pca_components = 10,
                          feature_selection = True,
                          train_size = 0.8, # Tamanho do dataframe de validação para o Pycaret
                          normalize = True, normalize_method = 'robust', # Normalizar as features numéricas usando minmax
                          feature_interaction = True, # Feature Engineering
                          feature_ratio = True, # Feature Engineering
                          combine_rare_levels = True,
                          remove_multicollinearity = True,
                          profile = True) # Data profiling

    return ml

In [ ]:
def pycaret_classificacao(df, target):
    print(f'Modelo: ml_{target}')
    print(f'Este é um problema de Classificação')

    ml = classification.setup(data = df, 
                            target = target, 
                            session_id = 20111974,
                            #ignore_features = ['id'],
                            #numeric_features = l_features_numericas, # Necesssário para forçar o Pycaret a considerar estas features como numéricas
                            #ignore_low_variance = True,
                            #pca = True, pca_components = 10,
                            feature_selection = True,
                            train_size = 0.8, # Tamanho do dataframe de validação para o Pycaret
                            normalize = True, normalize_method = 'robust', # Normalizar as features numéricas usando minmax
                            feature_interaction = True, # Feature Engineering
                            feature_ratio = True, # Feature Engineering
                            combine_rare_levels = True,
                            remove_multicollinearity = True,
                            profile = True, # Data profiling
                            fix_imbalance = True) # para balancear a amostra
                                  
    #classification.compare_models()
    return ml

A seguir, vamos chamar a função usando os específicos dataframes previamente construídos:

### TotalCharges2 (0.14% de missing values) --> Regressão
* Atenção aos pressupostos da Regressão:
    * O melhor modelo possui o maior Adjusted R2;
    * Certifique-se de que os erros do modelo (também chamados de resíduos) estejam igualmente distribuidos em torno de 0 (zero);
    * Certifique-se de que os erros do modelo sejam pequenos. Para isso, veja o gráfico de envelope, onde os erros estejam entre faixas. O ideal é que estas faixas sejam mais estreitas possíveis, o que significa que o desvio-padrão é pequeno. **Veja o gráfico residual plot**.

#### Setup

In [ ]:
mostra_dados(df_TotalCharges2_sem_mv)

### Setup

In [ ]:
setup_TotalCharges2 = pycaret_regressao(df = df_TotalCharges2_sem_mv, target = 'TotalCharges2')

#### Modelos de Machine Learning

A seguir, a construção dos modelos usando Pycaret:

In [ ]:
ml_TotalCharges2 = regression.compare_models(fold = 10, sort = 'R2', include = ['lasso', 'br', 'ridge', 'gbr', 'lightgbm', 'catboost', 'xgboost'])

O Melhor modelo (baseline) e parâmetros foram:

In [ ]:
ml_TotalCharges2.get_params

#### Ajustar o melhor modelo (baseado nas indicações do Pycaret):
* Conforme vimos anteriormente, o melhor modelo foi lasso. Portanto, vamos focar neste modelo de Machine Learning daqui para frente para fazermos o tratamento dos missing values.

In [ ]:
ml_lasso_TotalCharges2 = regression.create_model('lasso')

#### Parameter Tunning

In [ ]:
ml_lasso_pt_TotalCharges2 = regression.tune_model(ml_lasso_TotalCharges2, n_iter = 50)

#### Gráficos

In [ ]:
regression.evaluate_model(ml_lasso_pt_TotalCharges2)

#### Interpretação do modelo - SOMENTE PARA CLASSIFICAÇÃO

#### Predição dos missing values

In [ ]:
regression.predict_model(ml_lasso_pt_TotalCharges2)

Abaixo, a aplicação do modelo para estimar (predict) dos missing values da variável sob análise. O PyCaret constroi um pipeline com todas as transformações. Desta forma, as mesmas transformações feitas no dataframe de treinamento serão aplicadas no dataframe teste e apresentar os resultados.

In [ ]:
df_T3 = regression.predict_model(ml_lasso_pt_TotalCharges2, data = df_T2)

In [ ]:
df_V3 = regression.predict_model(ml_lasso_pt_TotalCharges2, data = df_V2) # Gerar as mesmas transformações em df_V para que o modelo seja aplicado!!!

Compare as colunas TotalCharges2 e Label abaixo:

In [ ]:
df_T3.isna().sum()

In [ ]:
df_T3.head()

A seguir, Boxplot da variável/feature original:

In [ ]:
sns.boxplot(x = df_T3["Churn"], y = df_T3["TotalCharges2"])
plt.show()

A seguir, Boxplot da variável estimada pelo modelo de Machine Learning:

In [ ]:
sns.boxplot(x = df_T3["Churn"], y = df_T3["Label"])

A seguir, as médias da variável original:

In [ ]:
df_T3[['TotalCharges2']].describe()

A seguir, as médias da variável estimada:

In [ ]:
df_T3[['Label']].describe()

### **Conclusão**
* Temos um modelo com R2 satisfatório. Na sequência, vamos fazer o tratamento dos missing values.

In [ ]:
df_T4 = df_T3.copy()
df_V4 = df_V3.copy()

df_T4.head()

A seguir, tratamento das instâncias missing values da variável.

In [ ]:
df_T4['TotalCharges3'] = df_T4['TotalCharges2']
df_T4['TotalCharges3'] = np.where(df_T4['TotalCharges2'].isna(), df_T4['Label'], df_T4['TotalCharges2'])

In [ ]:
df_V4['TotalCharges3'] = df_V4['TotalCharges2']
df_V4['TotalCharges3'] = np.where(df_V4['TotalCharges2'].isna(), df_V4['Label'], df_V4['TotalCharges2'])

Vamos comparar as médias mais uma vez:

#### Treinamento

In [ ]:
df_T4['TotalCharges2'].describe()

In [ ]:
df_T4['TotalCharges3'].describe()

#### Validação

In [ ]:
df_V4['TotalCharges2'].describe()

In [ ]:
df_V4['TotalCharges3'].describe()

Deletar a coluna dos dataframes de treinamento e validação

In [ ]:
df_T4 = df_T4.drop(columns = ['TotalCharges2', 'Label'], axis = 1)
df_V4 = df_V4.drop(columns = ['TotalCharges2', 'Label'], axis = 1)

In [ ]:
df_T4.head()

#### Salvar o modelo para uso futuro (se necessário)

In [ ]:
regression.save_model(ml_lasso_pt_TotalCharges2, 'ml_lasso_pt_TotalCharges2')

#### Carregar o modelo (que foi previamente salvo)

In [ ]:
ml_lasso_pt_TotalCharges2_load = regression.load_model(model_name = 'ml_lasso_pt_TotalCharges2')

### tenure (8.18% de missing values) --> Regressão

#### Setup

In [ ]:
setup_tenure = pycaret_regressao(df = df_tenure_sem_mv, target = 'tenure')

Olhando para o output acima, há alguma coisa que gostaríamos de alterar/mudar? Por exemplo, construir binning para variáveis numéricas?

**Por exemplo**: eu gostaria de construir 5 bins para a variável MonthlyCharges.

#### Modelos de Machine Learning

A seguir, a construção dos modelos usando Pycaret:

In [ ]:
ml_tenure = regression.compare_models(fold = 10, sort = 'R2') #, include = ['lasso', 'br', 'ridge', 'gbr', 'lightgbm', 'catboost', 'xgboost'])

### **Conclusão**
* Temos um modelo insatisfatório. Sugestões:
    * Binarizar a variável

In [ ]:
df_T5 = df_T4.copy()
df_V5 = df_V4.copy()

df_T5.head()

**Insight**: Na próxima versão, construir os buckets (classes) da variável tenure usando Decision Tree.

In [ ]:
df_T5['tenure_bin'] = pd.qcut(df_T5['tenure'], q = [0, .2, .4, .6, .8, 1], precision = 0, labels = ['0_20', '20_40', '40_60', '60_80', '80_100'])

# Lembre-se que esta transformação precisa ser feita no dataframe de teste:
df_V5['tenure_bin'] = pd.qcut(df_V5['tenure'], q = [0, .2, .4, .6, .8, 1], precision = 0, labels = ['0_20', '20_40', '40_60', '60_80', '80_100'])

df_T5.head()

In [ ]:
df_T5 = df_T5.drop(columns = ['tenure'], axis = 1)
df_V5 = df_V5.drop(columns = ['tenure'], axis = 1)

A seguir, construção das variáveis dummies:

In [ ]:
df_T5 = pd.get_dummies(df_T5, columns = ['tenure_bin'], dummy_na = True)
df_V5 = pd.get_dummies(df_V5, columns = ['tenure_bin'], dummy_na = True)
df_T5.head()

In [ ]:
df_V5.head()

### PaymentMethod (1.76% de missing values) --> Classificação


In [ ]:
df_T6 = df_T5.copy()
df_V6 = df_V5.copy()

df_PaymentMethod_sem_mv_copia = df_PaymentMethod_sem_mv.copy()

In [ ]:
#df_PaymentMethod_sem_mv = df_PaymentMethod_sem_mv_copia.copy()

#### Setup
* Precisamos transformar a variável-target para numérica.

In [ ]:
Counter(df_T6['PaymentMethod'])

In [ ]:
label_PaymentMethod = {'Bank transfer (automatic)': 1, 
                       'Credit card (automatic)': 2, 
                       'Electronic check': 3, 
                       'Mailed check': 4,
                       np.nan: 0} # Nossos missing values são os valores '0', ok?

In [ ]:
# Aplica o dicionário:
df_T6['PaymentMethod2'] = df_T6['PaymentMethod'].map(label_PaymentMethod).astype(np.int64)
df_V6['PaymentMethod2'] = df_V6['PaymentMethod'].map(label_PaymentMethod).astype(np.int64)

# Aplica o dicionário no dataframe de treinamento que usaremos para estimar os missing values:
df_PaymentMethod_sem_mv['PaymentMethod2'] = df_PaymentMethod_sem_mv['PaymentMethod'].map(label_PaymentMethod).astype(np.int64)

In [ ]:
Counter(df_T6['PaymentMethod2'])

In [ ]:
Counter(df_PaymentMethod_sem_mv['PaymentMethod2'])

In [ ]:
df_T6.head()

In [ ]:
#df_T6 = df_T6.drop(columns = ['PaymentMethod'])
#df_V6 = df_V6.drop(columns = ['PaymentMethod'])

# Dropa a coluna original do dataframe de treinamento
df_PaymentMethod_sem_mv = df_PaymentMethod_sem_mv.drop(columns = ['PaymentMethod'])

In [ ]:
setup_PaymentMethod2 = pycaret_classificacao(df = df_PaymentMethod_sem_mv, target = 'PaymentMethod2')

Olhando para o output acima, há alguma coisa que gostaríamos de alterar/mudar? Por exemplo, construir binning para variáveis numéricas?

**Por exemplo**: eu gostaria de construir 5 bins para a variável MonthlyCharges.

#### Modelos de Machine Learning

A seguir, a construção dos modelos usando Pycaret:

In [ ]:
ml_PaymentMethod2 = classification.compare_models(fold = 10, sort = 'Accuracy') #, include = ['lightgbm', 'ridge', 'lr', 'gbc', 'xgboost', 'catboost', 'nb'])

### **Conclusão**
* Não me agrada um modelo com tão baixa acurácia.
* Portanto, vou construir variáveis dummies para esta coluna.

### Segunda tentativa com PaymentMethod usando 2 categorias (1.76% de missing values) --> Classificação

* Tínhamos feito o seguinte:
```
label_PaymentMethod = {'Bank transfer (automatic)': 1, 
                       'Credit card (automatic)': 2, 
                       'Electronic check': 3, 
                       'Mailed check': 4,
                       np.nan: 0} # Nossos missing values são os valores '0', ok?
```


In [ ]:
df_T7 = df_T6.copy()
df_V7 = df_V6.copy()
df_PaymentMethod_sem_mv_copia = df_PaymentMethod_sem_mv.copy()

In [ ]:
#df_PaymentMethod_sem_mv = df_PaymentMethod_sem_mv_copia.copy()

Portanto, o nosso dicionário passa a ser:

In [ ]:
d_PaymentMethod = {1: 1, 
                   2: 1, 
                   3: 2, 
                   4: 2, 
                   0: 0} # Neste caso, nossos missing values serão 0!!!!

In [ ]:
d_PaymentMethod

Aplica o dicionário:

In [ ]:
Counter(df_PaymentMethod_sem_mv['PaymentMethod2'])

In [ ]:
Counter(df_T7['PaymentMethod2'])

In [ ]:
Counter(df_V7['PaymentMethod2'])

In [ ]:
df_PaymentMethod_sem_mv['PaymentMethod3'] = df_PaymentMethod_sem_mv['PaymentMethod2'].map(d_PaymentMethod)

df_T7['PaymentMethod3'] = df_T7['PaymentMethod2'].map(d_PaymentMethod)
df_V7['PaymentMethod3'] = df_V7['PaymentMethod2'].map(d_PaymentMethod)

In [ ]:
Counter(df_PaymentMethod_sem_mv['PaymentMethod3'])

#### Setup

In [ ]:
#df_T7 = df_T7.drop(columns = ['PaymentMethod2'])
#df_V7 = df_V7.drop(columns = ['PaymentMethod2'])

# Dropa a coluna original do dataframe de treinamento
df_PaymentMethod_sem_mv = df_PaymentMethod_sem_mv.drop(columns = ['PaymentMethod2'])

In [ ]:
setup_PaymentMethod3 = pycaret_classificacao(df = df_PaymentMethod_sem_mv, target = 'PaymentMethod3')

#### Modelos de Machine Learning

A seguir, a construção dos modelos usando Pycaret:

In [ ]:
ml_PaymentMethod3 = classification.compare_models(fold = 10, sort = 'Accuracy') #, include = ['lightgbm', 'ridge', 'lr', 'gbc', 'xgboost', 'catboost', 'nb'])

### **Conclusão**
* O modelo com 2 classes não resultou. Portanto, vamos avançar para a sugestão de criar variáveis dummies para PaymentMethod.

In [ ]:
df_T8 = df_T7.copy()
df_V8 = df_V7.copy()

In [ ]:
df_T8.head()

In [ ]:
df_T8 = pd.get_dummies(df_T8, columns = ['PaymentMethod'], dummy_na = True)
df_V8 = pd.get_dummies(df_V8, columns = ['PaymentMethod'], dummy_na = True)

df_T8.head()

### Dependents (3.87% de missing values) --> Classificação

In [ ]:
df_Dependents_sem_mv.head()

#### Setup

In [ ]:
setup_Dependents = pycaret_classificacao(df = df_Dependents_sem_mv, target = 'Dependents')

#### Modelos de Machine Learning

A seguir, a construção dos modelos usando Pycaret:

In [ ]:
ml_Dependents = classification.compare_models(fold = 10) # , sort = 'R2', include = ['lasso', 'br', 'ridge', 'gbr', 'lightgbm', 'catboost', 'xgboost'])

O Melhor modelo (baseline) e parâmetros foram:

In [ ]:
ml_Dependents.get_params

#### Ajustar o melhor modelo (baseado nas indicações do Pycaret):
* Conforme vimos anteriormente, o melhor modelo foi lasso. Portanto, vamos focar neste modelo de Machine Learning daqui para frente para fazermos o tratamento dos missing values.

In [ ]:
ml_gbc_Dependents = classification.create_model('gbc')

#### Parameter Tunning

In [ ]:
ml_gbc_pt_Dependents = classification.tune_model(ml_gbc_Dependents, n_iter = 50)

#### Gráficos

In [ ]:
classification.evaluate_model(ml_gbc_pt_Dependents)

#### Interpretação do modelo - SOMENTE PARA CLASSIFICAÇÃO

#### Predição dos missing values

In [ ]:
classification.predict_model(ml_gbc_pt_Dependents)

Abaixo, a aplicação do modelo para estimar (predict) dos missing values da variável sob análise. O PyCaret constroi um pipeline com todas as transformações. Desta forma, as mesmas transformações feitas no dataframe de treinamento serão aplicadas no dataframe teste e apresentar os resultados.

In [ ]:
df_T9 = classification.predict_model(ml_gbc_pt_Dependents, data = df_T8)

In [ ]:
df_V9 = classification.predict_model(ml_gbc_pt_Dependents, data = df_V8) # Gerar as mesmas transformações em df_V para que o modelo seja aplicado!!!

### **Conclusão**
* Temos um modelo com acurácia satisfatória. Na sequência, vamos fazer o tratamento dos missing values.

A seguir, tratamento das instâncias missing values da variável.

In [ ]:
df_T9[['Dependents', 'Label', 'Score']].head()

In [ ]:
df_T9['Dependents2'] = df_T9['Dependents']
df_T9['Dependents2'] = np.where(df_T9['Dependents'].isna(), df_T9['Label'], df_T9['Dependents'])

In [ ]:
df_V9['Dependents2'] = df_V9['Dependents']
df_V9['Dependents2'] = np.where(df_V9['Dependents'].isna(), df_V9['Label'], df_V9['Dependents'])

In [ ]:
df_T9.isna().sum()

Deletar a coluna dos dataframes de treinamento e validação

In [ ]:
df_T9 = df_T9.drop(columns = ['Dependents', 'Label', 'Score'], axis = 1)
df_V9 = df_V9.drop(columns = ['Dependents', 'Label', 'Score'], axis = 1)

In [ ]:
df_T9.isna().sum()

#### Salvar o modelo para uso futuro (se necessário)

In [ ]:
classification.save_model(ml_gbc_pt_Dependents, 'ml_gbc_pt_Dependents')

#### Carregar o modelo (que foi previamente salvo)

In [ ]:
ml_gbc_pt_Dependents_load = classification.load_model(model_name = 'ml_gbc_pt_Dependents')

__
# Final dos tratamentos de missing values

In [ ]:
l_features_numericas_bin = ['MonthlyCharges', 'tenure', 'TotalCharges2']
for coluna in l_features_numericas_bin:
    df_T[coluna+'_bin'] = pd.cut(df_T[coluna], bins = 5, labels = ['A', 'B', 'C', 'D', 'E'])
    df_T = pd.get_dummies(df_T, columns = [coluna+'_bin'], dummy_na = True) 
    df_T = df_T.drop(columns = [coluna], axis = 1)   
    #collections.Counter(df_T[coluna+'_Bin'])

## Tratamento das variáveis numéricas
* **Proposta**: Construir 5 categorias/classes para as variáveis numéricas do dataframe:
    * MonthlyCharges
    * tenure
    * TotalCharges2


In [ ]:
l_features_numericas_bin = ['MonthlyCharges', 'tenure', 'TotalCharges2']
for coluna in l_features_numericas_bin:
    df_T[coluna+'_bin'] = pd.cut(df_T[coluna], bins = 5, labels = ['A', 'B', 'C', 'D', 'E'])
    df_T = pd.get_dummies(df_T, columns = [coluna+'_bin'], dummy_na = True) 
    df_T = df_T.drop(columns = [coluna], axis = 1)   
    #collections.Counter(df_T[coluna+'_Bin'])


In [ ]:

l_features_uint8 = df_T.select_dtypes(np.uint8)
l_features_uint8

In [ ]:
for coluna in l_features_uint8:
    df_T[coluna] = np.int64(df_T[coluna])

In [ ]:
df_T.dtypes

Abaixo, nova visualização após categorizarmos as variáveis numéricas:

In [ ]:
mostra_dados(df_T)

## Tratamento dos Outliers
* Não há Outliers. Portanto, avançar para Feature Selection

## Feature Selection



In [ ]:
df_T10 = df_T3.copy()
df_V10 = df_V3.copy()

#### Setup

In [ ]:
setup_Churn = classification.setup(data = df_T10, 
                                   target = 'Churn', 
                                   session_id = 20111974,
                                   imputation_type = 'iterative', 
                                   iterative_imputation_iters = 20, 
                                   categorical_iterative_imputer = 'knn',
                                   numeric_iterative_imputer = 'knn',
                                   feature_selection = True,
                                   train_size = 0.8, # Tamanho do dataframe de validação para o Pycaret
                                   normalize = True, normalize_method = 'robust', # Normalizar as features numéricas usando minmax
                                   feature_interaction = True, # Feature Engineering
                                   feature_ratio = True, # Feature Engineering
                                   combine_rare_levels = True,
                                   remove_multicollinearity = True,
                                   profile = True, # Data profiling
                                   fix_imbalance = True # para balancear a amostra
                                   )

#### Modelos de Machine Learning

A seguir, a construção dos modelos usando Pycaret:

In [ ]:
ml_Churn = classification.compare_models()

O Melhor modelo (baseline) e parâmetros foram:

In [ ]:
ml_Churn.get_params

#### Ajustar o melhor modelo (baseado nas indicações do Pycaret):
* Conforme vimos anteriormente, o melhor modelo foi lasso. Portanto, vamos focar neste modelo de Machine Learning daqui para frente para fazermos o tratamento dos missing values.

In [ ]:
ml_gbc_Churn = classification.create_model('gbc')

#### Parameter Tunning

In [ ]:
ml_gbc_pt_Churn = classification.tune_model(ml_gbc_Churn, n_iter = 100, optimize = 'Accuracy')

In [ ]:
optimize_threshold(gbc, true_negative = ???, false_negative = ???) # Precisa descobrir os custos de um falso positivo e falso negativo. A partir disso, podemos mudar o threshold.

#### Gráficos

In [ ]:
classification.evaluate_model(ml_gbc_pt_Churn)

#### Interpretação do modelo - SOMENTE PARA CLASSIFICAÇÃO

#### Predição dos missing values

In [ ]:
classification.predict_model(ml_gbc_pt_Churn) # predict_model(xgboost, probability_threshold=0.2)

Abaixo, a aplicação do modelo para estimar (predict) dos missing values da variável sob análise. O PyCaret constroi um pipeline com todas as transformações. Desta forma, as mesmas transformações feitas no dataframe de treinamento serão aplicadas no dataframe teste e apresentar os resultados.

In [ ]:
df_T11 = classification.predict_model(ml_gbc_pt_Churn, data = df_T10)

In [ ]:
df_V11 = classification.predict_model(ml_gbc_pt_Churn, data = df_V10) # Gerar as mesmas transformações em df_V para que o modelo seja aplicado!!!

### **Conclusão**
* Temos um modelo com acurácia satisfatória. Na sequência, vamos fazer o tratamento dos missing values.

A seguir, tratamento das instâncias missing values da variável.

In [ ]:
df_T11[['Churn', 'Label', 'Score']].head()

In [ ]:
df_T11['Churn2'] = df_T11['Churn']
df_T11['Churn2'] = np.where(df_T11['Churn'].isna(), df_T11['Label'], df_T11['Churn'])

In [ ]:
df_V11['Churn2'] = df_V9['Churn']
df_V11['Churn2'] = np.where(df_V11['Churn'].isna(), df_V11['Label'], df_V11['Churn'])

In [ ]:
df_T11.isna().sum()

Deletar a coluna dos dataframes de treinamento e validação

In [ ]:
df_T11 = df_T11.drop(columns = ['Churn', 'Label', 'Score'], axis = 1)
df_V11 = df_V11.drop(columns = ['Churn', 'Label', 'Score'], axis = 1)

In [ ]:
df_T11.isna().sum()

#### Salvar o modelo para uso futuro (se necessário)

In [ ]:
classification.save_model(ml_gbc_pt_Dependents, 'ml_gbc_pt_Churn')

#### Carregar o modelo (que foi previamente salvo)

In [ ]:
ml_gbc_pt_Dependents_load = classification.load_model(model_name = 'ml_gbc_pt_Churn')